In [0]:
import datetime as dt
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.tree import DecisionTreeClassifier
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df1 = pd.read_csv('/content/drive/My Drive/SET2/datasets/preprocessed_accidents.csv', error_bad_lines=False, index_col='Accident_Index', warn_bad_lines=False);
print(df1.corr(method="pearson"));

                                         Longitude  ...  Road_Surface_Conditions
Longitude                                 1.000000  ...                -0.071987
Latitude                                 -0.430738  ...                 0.084356
Police_Force                             -0.379445  ...                 0.082636
Accident_Severity                         0.017307  ...                 0.010136
Number_of_Vehicles                        0.019948  ...                -0.042574
Number_of_Casualties                     -0.037849  ...                 0.027692
Day_of_Week                              -0.002509  ...                -0.011217
1st_Road_Class                           -0.057958  ...                 0.002278
1st_Road_Number                          -0.080044  ...                 0.015667
Road_Type                                -0.012452  ...                 0.014417
Speed_limit                              -0.062189  ...                 0.130650
2nd_Road_Number             

In [6]:
accidents = pd.read_csv('/content/drive/My Drive/SET2/datasets/preprocessed_accidents.csv', index_col='Accident_Index')
vehicles = pd.read_csv('/content/drive/My Drive/SET2/datasets/Vehicles0515.csv', error_bad_lines=False,warn_bad_lines=False)
vehicles.count()
vehicles.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
0,200501BS00001,1,9,0,18,0,0,0,0,0,0,1,1,15,2,74,10,-1,-1,-1,7,1
1,200501BS00002,1,11,0,4,0,3,0,0,0,0,4,1,1,1,42,7,8268,2,3,-1,-1
2,200501BS00003,1,11,0,17,0,0,0,4,0,0,4,1,1,1,35,6,8300,2,5,2,1
3,200501BS00003,2,9,0,2,0,0,0,0,0,0,3,1,15,1,62,9,1762,1,6,1,1
4,200501BS00004,1,9,0,18,0,0,0,0,0,0,1,1,15,2,49,8,1769,1,4,2,1


In [7]:
vehicles=vehicles[vehicles.Age_of_Driver!=-1]
vehicles=vehicles[vehicles["Engine_Capacity_(CC)"]!=-1]
vehicles=vehicles[vehicles.Age_of_Vehicle!=-1]
print(vehicles.count())
vehicles_new=pd.DataFrame(vehicles[['Accident_Index','Vehicle_Type','Sex_of_Driver','Age_of_Driver','Engine_Capacity_(CC)','Age_of_Vehicle']])
vehicles_new.head()
vehicles_new.to_csv('/content/drive/My Drive/SET2/datasets/preprocessed_Vehicles0515.csv')

Accident_Index                      1967520
Vehicle_Reference                   1967520
Vehicle_Type                        1967520
Towing_and_Articulation             1967520
Vehicle_Manoeuvre                   1967520
Vehicle_Location-Restricted_Lane    1967520
Junction_Location                   1967520
Skidding_and_Overturning            1967520
Hit_Object_in_Carriageway           1967520
Vehicle_Leaving_Carriageway         1967520
Hit_Object_off_Carriageway          1967520
1st_Point_of_Impact                 1967520
Was_Vehicle_Left_Hand_Drive?        1967520
Journey_Purpose_of_Driver           1967520
Sex_of_Driver                       1967520
Age_of_Driver                       1967520
Age_Band_of_Driver                  1967520
Engine_Capacity_(CC)                1967520
Propulsion_Code                     1967520
Age_of_Vehicle                      1967520
Driver_IMD_Decile                   1967520
Driver_Home_Area_Type               1967520
dtype: int64


In [8]:
corr_matrix = accidents.corr()
print(corr_matrix["Accident_Severity"].sort_values(ascending=False))

Accident_Severity                          1.000000
Number_of_Vehicles                         0.074951
2nd_Road_Number                            0.025427
Weather_Conditions                         0.021433
Longitude                                  0.017307
1st_Road_Class                             0.011956
Road_Surface_Conditions                    0.010136
Pedestrian_Crossing-Physical_Facilities    0.008103
Pedestrian_Crossing-Human_Control          0.004867
Day_of_Week                                0.002552
1st_Road_Number                           -0.004735
Latitude                                  -0.032696
Police_Force                              -0.036113
Road_Type                                 -0.039005
Light_Conditions                          -0.063311
Speed_limit                               -0.080690
Number_of_Casualties                      -0.084156
Name: Accident_Severity, dtype: float64


In [9]:
vehicles_new = pd.read_csv('/content/drive/My Drive/SET2/datasets/preprocessed_Vehicles0515.csv', index_col='Accident_Index',error_bad_lines=False,warn_bad_lines=False)
accidents = accidents.join(vehicles_new, how='inner')
#accidents = pd.concat([accidents, vehicles_new], axis=1, join='inner')
accidents.head()
accidents['Date_time'] =  accidents['Date'] +' '+ accidents['Time']
for col in accidents.columns:
    print(col)
    accidents['Date_time'] = pd.to_datetime(accidents.Date_time)

accidents.drop(['Date','Time'],axis =1 , inplace=True)
accidents.dropna(inplace=True)
accident_ml = accidents.drop('Accident_Severity' ,axis=1)
accident_ml = accident_ml[['Age_of_Driver' ,'Vehicle_Type', 'Age_of_Vehicle','Engine_Capacity_(CC)','Day_of_Week' , 'Weather_Conditions' , 'Road_Surface_Conditions'
                          , 'Light_Conditions', 'Sex_of_Driver' ,'Speed_limit']]

Longitude
Latitude
Police_Force
Accident_Severity
Number_of_Vehicles
Number_of_Casualties
Date
Day_of_Week
Time
1st_Road_Class
1st_Road_Number
Road_Type
Speed_limit
2nd_Road_Number
Pedestrian_Crossing-Human_Control
Pedestrian_Crossing-Physical_Facilities
Light_Conditions
Weather_Conditions
Road_Surface_Conditions
Unnamed: 0
Vehicle_Type
Sex_of_Driver
Age_of_Driver
Engine_Capacity_(CC)
Age_of_Vehicle
Date_time


In [10]:
corr_matrix1= accidents.corr()
print(print(corr_matrix1["Accident_Severity"].sort_values(ascending=False)))
print(accident_ml.columns)
accident_ml.head()

Accident_Severity                          1.000000
Sex_of_Driver                              0.070637
2nd_Road_Number                            0.027521
Weather_Conditions                         0.022946
Number_of_Vehicles                         0.020199
Pedestrian_Crossing-Physical_Facilities    0.019625
1st_Road_Class                             0.018602
Road_Surface_Conditions                    0.011632
Vehicle_Type                               0.008394
Longitude                                  0.007887
Pedestrian_Crossing-Human_Control          0.006071
Day_of_Week                                0.004186
1st_Road_Number                           -0.001897
Unnamed: 0                                -0.005893
Age_of_Vehicle                            -0.012951
Engine_Capacity_(CC)                      -0.018105
Age_of_Driver                             -0.024606
Latitude                                  -0.025469
Police_Force                              -0.035530
Road_Type   

,Age_of_Driver,Vehicle_Type,Age_of_Vehicle,Engine_Capacity_(CC),Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Sex_of_Driver,Speed_limit
Accident_Index,,,,,,,,,,
200501BS00002,42,11,3,8268,4,1,1,4,1,30
200501BS00003,35,11,5,8300,5,1,1,4,1,30
200501BS00003,62,9,6,1762,5,1,1,4,1,30
200501BS00004,49,9,4,1769,6,1,1,1,2,30
200501BS00005,49,3,10,85,2,1,2,7,1,30


In [11]:
X_train, X_test, y_train, y_test = train_test_split(accident_ml.values,
                                              accidents['Accident_Severity'].values,test_size=0.20, random_state=99)
print(accident_ml.head())
accident_ml.to_csv("/content/drive/My Drive/SET2/datasets/traintestdataset.csv")
print(accident_ml.columns)
print(X_train)
print(y_train)

                Age_of_Driver  Vehicle_Type  ...  Sex_of_Driver  Speed_limit
Accident_Index                               ...                            
200501BS00002              42            11  ...              1           30
200501BS00003              35            11  ...              1           30
200501BS00003              62             9  ...              1           30
200501BS00004              49             9  ...              2           30
200501BS00005              49             3  ...              1           30

[5 rows x 10 columns]
Index(['Age_of_Driver', 'Vehicle_Type', 'Age_of_Vehicle',
       'Engine_Capacity_(CC)', 'Day_of_Week', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Light_Conditions', 'Sex_of_Driver',
       'Speed_limit'],
      dtype='object')
[[18  9  9 ...  1  1 70]
 [24  9  1 ...  1  1 30]
 [42  9 10 ...  1  1 30]
 ...
 [31  9 12 ...  1  2 60]
 [48  9  7 ...  6  1 40]
 [75  9  8 ...  1  2 30]]
[3 3 3 ... 3 3 2]


In [0]:
random_forest = RandomForestClassifier(n_estimators=200)
random_forest.fit(X_train,y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_test, y_test)
acc_random_forest1 = round(random_forest.score(X_test, y_test) * 100, 2)

In [0]:
sk_report = classification_report(
    digits=6,
    y_true=y_test,
    y_pred=Y_pred)
print("Accuracy" , acc_random_forest1)
print(sk_report)
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 84.41
              precision    recall  f1-score   support

           1   0.056657  0.007271  0.012889      5501
           2   0.219332  0.056924  0.090389     48591
           3   0.866321  0.970538  0.915473    338947

    accuracy                       0.844107    393039
   macro avg   0.380770  0.344911  0.339584    393039
weighted avg   0.775002  0.844107  0.800836    393039



Predicted,1,2,3,All
Actual,,,,
1,40,387,5074,5501
2,138,2766,45687,48591
3,528,9458,328961,338947
All,706,12611,379722,393039


In [0]:
lr = LogisticRegression()
# Fit the model on the trainng data.
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test,
    y_pred=y_pred)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 86.23
              precision    recall  f1-score   support

           1   0.000000  0.000000  0.000000      5501
           2   0.200000  0.000021  0.000041     48591
           3   0.862374  0.999959  0.926084    338947

    accuracy                       0.862342    393039
   macro avg   0.354125  0.333326  0.308709    393039
weighted avg   0.768416  0.862342  0.798637    393039



Predicted,1,2,3,All
Actual,,,,
1,0,0,5501,5501
2,1,1,48589,48591
3,10,4,338933,338947
All,11,5,393023,393039


In [0]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree1 = round(decision_tree.score(X_test, y_test) * 100, 2)
sk_report = classification_report(
    digits=6,
    y_true=y_test,
    y_pred=Y_pred)
print("Accuracy", acc_decision_tree1)
print(sk_report)
### Confusion Matrix
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)


Accuracy 74.8
              precision    recall  f1-score   support

           1   0.037211  0.048900  0.042262      5501
           2   0.149981  0.179560  0.163443     48591
           3   0.869822  0.840795  0.855062    338947

    accuracy                       0.747964    393039
   macro avg   0.352338  0.356418  0.353589    393039
weighted avg   0.769176  0.747964  0.758182    393039



Predicted,1,2,3,All
Actual,,,,
1,269,1166,4066,5501
2,1281,8725,38585,48591
3,5679,48283,284985,338947
All,7229,58174,327636,393039


In [0]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
y_pred = gnb.predict(X_test) 
  
# accuracy on X_test 
#accuracy = gnb.score(X_test, y_test)
accuracy=round(decision_tree.score(X_test, y_test) * 100, 2)
print("Accuracy", accuracy)

sk_report = classification_report(
    digits=6,
    y_true=y_test,
    y_pred=Y_pred)
print(sk_report)
  
# creating a confusion matrix 
#cm = confusion_matrix(y_test, gnb_predictions) 
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 74.79
              precision    recall  f1-score   support

           1   0.035615  0.046719  0.040418      5501
           2   0.150021  0.179663  0.163509     48591
           3   0.869820  0.840780  0.855054    338947

    accuracy                       0.747933    393039
   macro avg   0.351819  0.355721  0.352994    393039
weighted avg   0.769156  0.747933  0.758157    393039



Predicted,1,2,3,All
Actual,,,,
1,414,36,5051,5501
2,1476,273,46842,48591
3,7911,716,330320,338947
All,9801,1025,382213,393039
